In [3]:
with open("input.txt") as f:
    inputs = f.read().split("\n\n")

# Part 1

In [39]:
import re


required_fields = {"byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"}

field_pattern = r"([bie]yr|hgt|[he]cl|pid):(\S+)"


def is_valid_passport(passport):
    return set(
        extracted_field.group(1)
        for extracted_field in re.finditer(
            field_pattern, 
            passport,
            flags=re.MULTILINE
        )
    ) == required_fields


In [40]:
sum(map(is_valid_passport, inputs))

213

# Part 2

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

- byr (Birth Year) - four digits; at least 1920 and at most 2002.
- iyr (Issue Year) - four digits; at least 2010 and at most 2020.
- eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
- hgt (Height) - a number followed by either cm or in:
- If cm, the number must be at least 150 and at most 193.
- If in, the number must be at least 59 and at most 76.
- hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
- ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
- pid (Passport ID) - a nine-digit number, including leading zeroes.
- cid (Country ID) - ignored, missing or not.

In [101]:
from functools import partial


s_in_range = lambda a, b: lambda m: re.match(r"\d+", m) and a <= int(m) <= b

hgt_cm_val = s_in_range(150, 193)
hgt_in_val = s_in_range(59, 76)

height_cm = lambda m: hgt_cm_val(m.rstrip("cm"))
height_in = lambda m: hgt_in_val(m.rstrip("in"))

validators = {
    "byr": s_in_range(1920, 2002),
    "iyr": s_in_range(2010, 2020),
    "eyr": s_in_range(2020, 2030),
    "hgt": lambda m: height_cm(m) if m.endswith("cm") else (height_in(m) if m.endswith("in") else False),
    "hcl": lambda m: bool(re.match(r"#[0-9a-f]{6}", m)),
    "ecl": lambda m: m in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"},
    "pid": lambda m: len(m) == 9 and int(m) is not None
}


def is_valid_passport_2(passport):
    fields = {
        f.group(1): f.group(2)
        for f in re.finditer(
            field_pattern,
            passport,
            flags=re.MULTILINE
        )
    }
    
    if set(required_fields) - set(fields) != set():
        return False
    
    for field, value in fields.items():
        try:
            assert bool(validators[field](value)) == True
        except:
            return False
        
    return True


In [102]:
sum(map(is_valid_passport, inputs))

213

In [103]:
sum(map(is_valid_passport_2, inputs))

147